In [1]:
## 同一ディレクトリ内にhttps://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit?resourcekey=0-wjGZdNAUop6WykTtMip30g からダウンロードした学習済み単語ベクトルを配置しているものとする

from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00359/NewsAggregatorDataset.zip

--2023-01-30 12:55:37--  https://archive.ics.uci.edu/ml/machine-learning-databases/00359/NewsAggregatorDataset.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29224203 (28M) [application/x-httpd-php]
Saving to: ‘NewsAggregatorDataset.zip’

NewsAggregatorDatas 100%[===================>]  27.87M  27.0MB/s    in 1.0s    

2023-01-30 12:55:39 (27.0 MB/s) - ‘NewsAggregatorDataset.zip’ saved [29224203/29224203]



In [3]:
!unzip NewsAggregatorDataset.zip

Archive:  NewsAggregatorDataset.zip
  inflating: 2pageSessions.csv       
   creating: __MACOSX/
  inflating: __MACOSX/._2pageSessions.csv  
  inflating: newsCorpora.csv         
  inflating: __MACOSX/._newsCorpora.csv  
  inflating: readme.txt              
  inflating: __MACOSX/._readme.txt   


In [92]:
import random

CATEGORIES = [
    'b',
    't',
    't',
    'm'
]

with open('./newsCorpora.csv') as f:
   all_news = [news for news in f.readlines() if news.split('\t')[3] in TARGET_PUBLISHERS]

random.shuffle(all_news)

train_data_num = len(all_news)  * 8 // 10
valid_data_num = len(all_news)  * 9 // 10
train_data = all_news[: train_data_num]
valid_data = all_news[train_data_num : valid_data_num]
test_data = all_news[valid_data_num :]

with open('./train.txt', mode='w') as f:
  f.writelines('\t'.join(train_data))

with open('./valid.txt', mode='w') as f:
  f.writelines('\t'.join(valid_data))

with open('./test.txt', mode='w') as f:
  f.writelines('\t'.join(test_data))

print(f'学習データ事例数: {len(train_data)}')
print(f'評価データ事例数: {len(test_data)}')

学習データ事例数: 10684
評価データ事例数: 1336


In [31]:
# 80

def calculate_id(filepath):
  word_dic = {}
  result = {}
  with open(filepath, mode='r') as f:
    data = f.readlines()
    for row in data:
      for word in row.split('\t')[2].strip("'").split(' '):
        if not word in word_dic:
          word_dic[word] = 0
        word_dic[word] += 1
  sorted_words = sorted([(key, value) for key, value in word_dic.items()], key=lambda x: x[1], reverse=True)
  i = 0
  for word in sorted_words:
    if word[1] > 1:
      i += 1
      result[word[0]] = i
    else:
      result[word[0]] = 0
  return result, i

words, T = calculate_id('./train.txt')

def get_id_by_word(word):
  try:
    return words[word]
  except:
    return 0


In [93]:
# 81
## TODO: 
import numpy as np
from scipy.special import softmax

def vector_emb(id, dw):
  if id > dw:
    id = 0
  x_class = np.array([id])
  return np.identity(dw, dtype="int32")[x_class].T

def ReLU(x):
    return max(x, 0)

def RNN(W_hx, W_hh, x, h, b_h):
  w_x = np.dot(W_hx, x).T[0]
  w_h = np.dot(W_hh, h)
  vReLU = np.vectorize(ReLU)
  return vReLU(w_x + w_h + b_h)

def calculate_y(W_yh, h, b_y):
  return softmax(np.dot(W_yh, h) + b_y,)


def calculate_RNN(X, W_hx, W_hh, W_yh, b_h, b_y, dh, dw):
  h = np.zeros(dh)
  for x in X:
    h = RNN(W_hx, W_hh, vector_emb(x, dw), h, b_h)
  y = calculate_y(W_yh, h, b_y)
  return y

dw = 300
dh = 50
W_hx = np.ones((dh, dw))
W_hh = np.ones((dh, dh))
W_yh = np.ones((len(CATEGORIES), dh))
b_h = np.ones(dh)
b_y = np.ones(len(CATEGORIES))

words = [1,2,3,4,5]
calculate_RNN(words, W_hx, W_hh, W_yh, b_h, b_y, dh, dw)

array([0.25, 0.25, 0.25, 0.25])

In [102]:
# 82 

dw = 300
dh = 50
W_hx = np.ones((dh, dw))
W_hh = np.ones((dh, dh))
W_yh = np.ones((len(CATEGORIES), dh))
b_h = np.ones(dh)
b_y = np.ones(len(CATEGORIES))
eta = 10.0

with open('./test.txt', mode='r') as f:
    data = f.readlines()
    for row in data:
      try:
        X = [get_id_by_word(word) for word in row.split('\t')[2].strip("'").split(' ')]
        y = vector_emb(CATEGORIES.index(row.split('\t')[5]), len(CATEGORIES)).T[0]
        y_hat = calculate_RNN(words, W_hx, W_hh, W_yh, b_h, b_y, dh, dw)
        W_hx = W_hx + eta * (y_hat - y) * X
      except:
        continue



In [63]:
a = np.ones((2, 5))
b = np.ones((5, 1))
np.dot(a, b)

array([[5.],
       [5.]])